In [1]:
import re
import math
import sys
import numpy as np
import pandas as pd

from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from pprint import pprint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score, mean_absolute_error
from scipy import stats
from sklearn.model_selection import RepeatedKFold
import scipy
import sklearn

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

pd.set_option("display.max_rows", 1000000)
pd.set_option("display.max_columns", 1000000)
pd.set_option("display.width", 1000000)
pd.set_option("display.max_colwidth", 1000000)
pd.set_option("display.max_info_columns", 1000)
np.set_printoptions(threshold=sys.maxsize)

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))

import qgrid

In [2]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
        border-style: solid;
        border: 1px solid lightgray;
    }
    tr:nth-child(even) {
        background: #eee;
    }
    th {
       background-color: lemonchiffon;
       color: black;
}
</style>

In [3]:
df = pd.read_pickle("diabetic_data.pkl")
df.head(5)

race  gender      age weight  admission_type_id  discharge_disposition_id  admission_source_id  time_in_hospital payer_code                medical_specialty  num_lab_procedures  num_procedures  num_medications  number_outpatient  number_emergency  number_inpatient diag_1 diag_2 diag_3  number_diagnoses max_glu_serum A1Cresult metformin repaglinide nateglinide chlorpropamide glimepiride acetohexamide glipizide glyburide tolbutamide pioglitazone rosiglitazone acarbose miglitol troglitazone tolazamide examide citoglipton insulin glyburide-metformin glipizide-metformin glimepiride-pioglitazone metformin-rosiglitazone metformin-pioglitazone change diabetesMed readmitted  readmitted_less_30  imputed_age
patient_nbr encounter_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
135         24437208      Caucasian  Female  [50-60)      ?                  2                         1                    1                 8          ?                       Cardiology                  77               6               33                  0                 0                 0    401    997    560                 8          None      None    Steady          No          No             No          No            No        No      Down          No           No            No       No       No           No         No      No          No  Steady                  No                  No                       No                      No                     No     Ch         Yes        <30                   1           55
            26264286      Caucasian  Female  [50-60)      ?                  1                         1                    7                 3          ?  Surgery-Cardiovascular/Thoracic                  31               1               14                  0                 0                 1    998     41    250                 5          None      None    Steady          No          No             No          No            No        No    Steady          No           No            No       No       No           No         No      No          No      No                  No                  No                       No                      No                     No     Ch         Yes        >30                   0           55
378         29758806      Caucasian  Female  [50-60)      ?                  3                         1                    1                 2          ?                    Surgery-Neuro                  49               1               11                  0                 0                 0    722    305    250                 3          None      None        No          No          No             No          No            No        No        No          No           No            No       No       No           No         No      No          No      No                  No                  No                       No                      No                     No     No          No         NO                   0           55
729         189899286     Caucasian  Female  [80-90)      ?                  1                         3                    7                 4         MC                 InternalMedicine                  68               2               23                  0                 0                 0    820    493   E880                 9       

In [4]:
df.shape

(101766, 50)

In [5]:
# dict.fromkeys(['Beer','Alcohol','Beverage'], 'Drink')

In [6]:
df.replace("?", np.nan, inplace=True)

In [7]:
drop_cols = df.columns[(df.isnull().sum() / len(df)) > 0.2]
print(drop_cols)

Index(['weight', 'payer_code', 'medical_specialty'], dtype='object')


In [8]:
df.drop(drop_cols, axis=1, inplace=True)
df.drop(["age"], axis=1, inplace=True)

df["race"].fillna("unknown", inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 101766 entries, (135, 24437208) to (189502619, 371459060)
Data columns (total 46 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   race                      101766 non-null  object
 1   gender                    101766 non-null  object
 2   admission_type_id         101766 non-null  int8  
 3   discharge_disposition_id  101766 non-null  int8  
 4   admission_source_id       101766 non-null  int8  
 5   time_in_hospital          101766 non-null  int8  
 6   num_lab_procedures        101766 non-null  int16 
 7   num_procedures            101766 non-null  int8  
 8   num_medications           101766 non-null  int8  
 9   number_outpatient         101766 non-null  int8  
 10  number_emergency          101766 non-null  int8  
 11  number_inpatient          101766 non-null  int8  
 12  diag_1                    101745 non-null  object
 13  diag_2                    101

In [10]:
discharge_codes_to_drop = [11, 19, 20, 21]
drop_idxs = df[df.discharge_disposition_id.isin(discharge_codes_to_drop)].index
df.drop(drop_idxs, inplace=True)

In [11]:
drop_idxs = df[df.gender == "Unknown/Invalid"].index
df.drop(drop_idxs, inplace=True)

In [12]:
df.shape

(100111, 46)

In [13]:
id_cols = ["admission_type_id", "discharge_disposition_id", "admission_source_id"]
df[id_cols] = df[id_cols].astype("str")
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 100111 entries, (135, 24437208) to (189502619, 371459060)
Data columns (total 46 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   race                      100111 non-null  object
 1   gender                    100111 non-null  object
 2   admission_type_id         100111 non-null  object
 3   discharge_disposition_id  100111 non-null  object
 4   admission_source_id       100111 non-null  object
 5   time_in_hospital          100111 non-null  int8  
 6   num_lab_procedures        100111 non-null  int16 
 7   num_procedures            100111 non-null  int8  
 8   num_medications           100111 non-null  int8  
 9   number_outpatient         100111 non-null  int8  
 10  number_emergency          100111 non-null  int8  
 11  number_inpatient          100111 non-null  int8  
 12  diag_1                    100090 non-null  object
 13  diag_2                    997

In [14]:
df.nunique()

race                          6
gender                        2
admission_type_id             8
discharge_disposition_id     23
admission_source_id          17
time_in_hospital             14
num_lab_procedures          118
num_procedures                7
num_medications              75
number_outpatient            39
number_emergency             33
number_inpatient             21
diag_1                      715
diag_2                      747
diag_3                      787
number_diagnoses             16
max_glu_serum                 4
A1Cresult                     4
metformin                     4
repaglinide                   4
nateglinide                   4
chlorpropamide                4
glimepiride                   4
acetohexamide                 2
glipizide                     4
glyburide                     4
tolbutamide                   2
pioglitazone                  4
rosiglitazone                 4
acarbose                      4
miglitol                      4
troglita

In [15]:
medicine_cols = df.loc[:, "metformin":"metformin-pioglitazone"].columns
diag_cols = df.loc[:, "diag_1":"diag_3"].columns

In [16]:
df["num_med_changes"] = df[medicine_cols].replace({"No": 0, "Steady": 0, "Up": 1, "Down": 1}).sum(axis=1)
df["num_meds"] = df[medicine_cols].replace({"No": 0, "Steady": 1, "Up": 1, "Down": 1}).sum(axis=1)
df.drop(medicine_cols, axis=1, inplace=True)

df.drop(diag_cols, axis=1, inplace=True)

In [17]:
# drop_cols = []

# for col in medicine_cols:
#     vc_df = df[col].value_counts(normalize=True)
#     if vc_df['No'] < .95:
#         print(vc_df)
#     else:
#         drop_cols.append(col)


# df.drop(drop_cols, axis=1, inplace=True)

In [18]:
df.nunique()

race                          6
gender                        2
admission_type_id             8
discharge_disposition_id     23
admission_source_id          17
time_in_hospital             14
num_lab_procedures          118
num_procedures                7
num_medications              75
number_outpatient            39
number_emergency             33
number_inpatient             21
number_diagnoses             16
max_glu_serum                 4
A1Cresult                     4
change                        2
diabetesMed                   2
readmitted                    3
readmitted_less_30            2
imputed_age                  10
num_med_changes               5
num_meds                      7
dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 100111 entries, (135, 24437208) to (189502619, 371459060)
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   race                      100111 non-null  object
 1   gender                    100111 non-null  object
 2   admission_type_id         100111 non-null  object
 3   discharge_disposition_id  100111 non-null  object
 4   admission_source_id       100111 non-null  object
 5   time_in_hospital          100111 non-null  int8  
 6   num_lab_procedures        100111 non-null  int16 
 7   num_procedures            100111 non-null  int8  
 8   num_medications           100111 non-null  int8  
 9   number_outpatient         100111 non-null  int8  
 10  number_emergency          100111 non-null  int8  
 11  number_inpatient          100111 non-null  int8  
 12  number_diagnoses          100111 non-null  int8  
 13  max_glu_serum             100